In [ ]:
import os

import pandas as pd
import torch
from ray import tune

from rayTune_common.configs import config5, config4, config3, config2, config1
from rayTune_common.constants import ins, outs, metric, mode
from rayTune_common.constants import rs_logdir_5, rs_logdir_4, rs_logdir_3, rs_logdir_2, rs_logdir_1
from rayTune_common.model import Net


# Data loaders


In [ ]:
path = "dataset/training_set.csv"
train_set = pd.read_csv(path, index_col=0)
path = "dataset/validation_set.csv"
val_set = pd.read_csv(path, index_col=0)
path = "dataset/test_set.csv"
test_set = pd.read_csv(path, index_col=0)


def prepare_data(
        input_cols: [],
        output_cols: [],
        train_batch_size: int
):
    """
    Prepares the dataset to be used for HPO
    Converts to torch tensors and dataset loaders
    :param input_cols: list of strings
    :param output_cols: list of strings
    :param train_batch_size: Batch size
    :return:
    :return: train_loader, x_val, y_val, val_loader, x_test, y_test
    """
    # INPUT_COLS = ['CHK', 'PWH', 'PDC', 'TWH', 'FGAS', 'FOIL']
    # OUTPUT_COLS = ['QTOT']

    # Get input and output tensors and convert them to torch tensors
    x_train = torch.from_numpy(train_set[input_cols].values).to(torch.float)
    y_train = torch.from_numpy(train_set[output_cols].values).to(torch.float)

    x_val = torch.from_numpy(val_set[input_cols].values).to(torch.float)
    y_val = torch.from_numpy(val_set[output_cols].values).to(torch.float)

    # Create dataset loaders
    # Here we specify the batch size and if the dataset should be shuffled
    train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)

    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=len(val_set), shuffle=False)

    # Get input and output as torch tensors
    x_test = torch.from_numpy(test_set[input_cols].values).to(torch.float)
    y_test = torch.from_numpy(test_set[output_cols].values).to(torch.float)

    return train_loader, x_val, y_val, val_loader, x_test, y_test


# Training function

In [ ]:
def train(config, checkpoint_dir=None):
    """
    :param config: hyperparameter configuration
    :param checkpoint_dir: local checkpoint dir. Leave blank to use ~/ray_results
    :return:
    """
    net = Net(
        len(ins),
        int(config["hidden_layers"]),
        int(config["hidden_layer_width"]),
        len(outs),
        dropout_value=config["dropout"]
    )

    net = net.to(net.device)

    # Define loss and optimizer
    criterion = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"])

    # The `checkpoint_dir` parameter gets passed by Ray Tune when a checkpoint
    # should be restored.
    if checkpoint_dir:
        checkpoint = os.path.join(checkpoint_dir, "checkpoint")
        model_state, optimizer_state = torch.load(checkpoint)
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    # Import training, validation and test data
    train_loader, x_valid, y_valid, val_loader, x_test, y_test = prepare_data(
        input_cols=ins,
        output_cols=outs,
        train_batch_size=64
    )

    # Train Network
    for epoch in range(100):
        # specify that we are in training mode
        net.train()

        for inputs, labels in train_loader:

            inputs, labels = inputs.to(net.device), labels.to(net.device)
            # Zero the parameter gradients (from last iteration)
            optimizer.zero_grad()

            # Forward propagation
            outputs = net(inputs)

            # Compute cost function
            batch_mse = criterion(outputs, labels)

            reg_loss = 0
            for param in net.parameters():
                reg_loss += param.pow(2).sum()

            cost = batch_mse + config["l2"] * reg_loss

            # Backward propagation to compute gradient
            cost.backward()

            # Update parameters using gradient
            optimizer.step()

        # Specify that we are in evaluation mode
        net.eval()

        # Evaluate model on validation data
        mse_val = 0
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(net.device), labels.to(net.device)
            mse_val += torch.sum(torch.pow(labels - net(inputs), 2)).item()
        mse_val /= len(val_loader.dataset)

        # Here we save a checkpoint. It is automatically registered with
        # Ray Tune and will potentially be passed as the `checkpoint_dir`
        # parameter in future iterations.
        with tune.checkpoint_dir(step=epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save(
                (net.state_dict(), optimizer.state_dict()), path)
        tune.report(mean_square_error=mse_val)

# Optimizer

In [ ]:
def optimize(config: {}, iterations: int, experiment_name: str, logdir: str):
    result = tune.run(
        tune.with_parameters(train),
        name=experiment_name,
        config=config,
        metric=metric,
        mode=mode,
        num_samples=iterations,
        verbose=1,
        checkpoint_score_attr="min-mean_square_error",
        keep_checkpoints_num=2,
        local_dir=logdir,
        resources_per_trial={"cpu": 1, "gpu": 0}
    )

# Experiment

In [ ]:
def experiment():
    for i in range(0, 15):
        print("Starting New Experiment")
        experiment_name = "rs_" + str(i).rjust(3, "0")
        optimize(
            config=config5,
            iterations=100,
            experiment_name=experiment_name,
            logdir=rs_logdir_5
        )

        print("Starting New Experiment")
        experiment_name = "rs_" + str(i).rjust(3, "0")
        optimize(
            config=config4,
            iterations=100,
            experiment_name=experiment_name,
            logdir=rs_logdir_4
        )

        print("Starting New Experiment")
        experiment_name = "rs_" + str(i).rjust(3, "0")
        optimize(
            config=config3,
            iterations=100,
            experiment_name=experiment_name,
            logdir=rs_logdir_3
        )

        print("Starting New Experiment")
        experiment_name = "rs_" + str(i).rjust(3, "0")
        optimize(
            config=config2,
            iterations=100,
            experiment_name=experiment_name,
            logdir=rs_logdir_2
        )

        print("Starting New Experiment")
        experiment_name = "rs_" + str(i).rjust(3, "0")
        optimize(
            config=config1,
            iterations=100,
            experiment_name=experiment_name,
            logdir=rs_logdir_1
        )

In [ ]:
experiment()